# Multivariate Regression

# Show the collections that have been generated

In [ ]:
from pymongo import MongoClient
from process_collection import *
#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'multivariate_random_5_2012'
client = MongoClient(mongoServer)
################################################

db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
#collectionNames

# Formats parameters to be used for multivariate regression

In [2]:
import numpy as np
import statsmodels.api as sm
from get_cso import * 
from process_collection import * 
collectionName = collection #from cell above
ratio = 0.05
numSubcatch = 7
results = volume_reduction_and_runoff_volume_vs_lid_number(collectionName, numSubcatch, db)
csoResults = cso_reduction(collectionName, ratio, numSubcatch, results) 

### regression
y = csoResults["csoReduction"]
x = results['numLists']
#print len(x[1])
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)
print "Exact r-squared:"
print reg_m(y, x).rsquared
print reg_m(y, x).summary()


[0.0, 35205287.01624048, 44218304.49409175, 26018578.59645629, 35830746.758375645, 30019482.354815483, 41598539.61605954, 37973711.52017975, 43177922.74911702, 31130070.27609539, 39537583.04696286, 40236352.63315201, 34550918.056511045, 44552396.4861244, 43392459.57440531, 39821410.4329083, 28674092.879568696, 40888722.049883485, 40841803.4803915, 52657177.10090411, 38732575.11573613, 22081828.525667787, 31665658.30290091, 43006473.719028115, 33601902.63476324, 54747716.13439262, 47727462.057263255, 38268371.44713676, 32439905.58783543, 36125103.72427058, 19587550.11868763, 27732254.268708944, 44655665.81277561, 49172391.071891785, 30133658.27962756, 37702938.38968873, 38031166.4020915, 43037880.68224692, 37651206.10557628, 21857455.564632416, 47934949.98855674, 48965892.81339502, 49408444.86490786, 34544427.95735979, 31395342.980231285, 41388479.88199222, 33163820.942004323, 33467919.789096117, 40847149.05998325, 36273244.95014417, 40500400.02811217, 42648225.63181245, 32315274.140219

In [3]:
### regression for total cso outflow
y = csoResults["cso"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)
print "Exact r-squared:"
print reg_m(y, x).rsquared
print reg_m(y, x).summary()


[1058860558.6021079, 1023655271.5858674, 1014642254.1080161, 1032841980.0056516, 1023029811.8437322, 1028841076.2472924, 1017262018.9860483, 1020886847.0819281, 1015682635.8529909, 1027730488.3260125, 1019322975.555145, 1018624205.9689559, 1024309640.5455968, 1014308162.1159835, 1015468099.0277026, 1019039148.1691996, 1030186465.7225392, 1017971836.5522244, 1018018755.1217164, 1006203381.5012038, 1020127983.4863718, 1036778730.0764401, 1027194900.299207, 1015854084.8830798, 1025258655.9673446, 1004112842.4677153, 1011133096.5448446, 1020592187.1549711, 1026420653.0142725, 1022735454.8778373, 1039273008.4834203, 1031128304.3333989, 1014204892.7893323, 1009688167.5302161, 1028726900.3224803, 1021157620.2124192, 1020829392.2000164, 1015822677.919861, 1021209352.4965316, 1037003103.0374755, 1010925608.6135511, 1009894665.7887129, 1009452113.7372, 1024316130.6447481, 1027465215.6218766, 1017472078.7201157, 1025696737.6601036, 1025392638.8130118, 1018013409.5421246, 1022587313.6519637, 10183

In [4]:
### regression for total volume outflow
y = csoResults["totalVol"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)

print "Exact r-squared:"
print reg_m(y, x).rsquared
print reg_m(y, x).summary()


[2131530540.1267557, 2076904513.351469, 2063345363.1942697, 2091690763.52114, 2076866878.855345, 2085941404.1608388, 2066809366.094567, 2072692539.9888296, 2065876077.718072, 2084165217.523212, 2071087815.8462195, 2068998643.5037415, 2078230223.822757, 2062929680.4225364, 2064356505.8307197, 2070673553.6252465, 2087910476.0066757, 2068029491.2702146, 2069290307.4824407, 2050225506.5740757, 2071887572.4524193, 2096627554.3886573, 2082214444.5253968, 2066102699.3234177, 2080168999.562581, 2047121145.3818772, 2057438834.3685145, 2072502933.4925523, 2080768673.952271, 2075840379.4593742, 2101669459.2790375, 2089258888.3600523, 2062120518.5585668, 2055609952.703843, 2085127374.722504, 2073462687.2007678, 2073019482.099858, 2064888168.8286836, 2074328421.9933515, 2097719809.7995703, 2057186299.4938288, 2055781246.8871675, 2054963918.539505, 2078054708.3819854, 2083399459.8804235, 2067217622.9540854, 2080189466.265297, 2079677051.3933449, 2067865044.0068388, 2076208120.3264828, 2069503450.686